# Todo-toepassing met AJAX

In deze versie gebruiken we JavaScript in de browser (in plaats van puur HTML form) en JSON (in plaats van HTML) voor de client-serverinteractie tussen de browser en de server.

Met JavaScript sturen we een verzoek naar de server; het antwoord, in de vorm van JSON-data, verwerken we ook met JavaScript in de DOM van de browser.
Van dit laatste geven we eerst een eenvoudige versie 

## Interface met de server: REST API

* `get /todos` - geeft de hele lijst met todo's
* `get /todos/<id>` - geeft een enkele todo
* `post /todos` - voor het aanmaken van een todo (met een beschrijving)
* `put /todos/<id>` - voor het aanpassen van een bestaande todo
* `delete /todos/<id>` - voor het verwijderen van een todo.

Dit is een voorbeeld van een REST API. Zie bijv.: https://restapitutorial.com

En een speciale URL, voor het testen van de API:

* `post /todos/reset` - brengt de database is een gedefinieerde (begin)toestand.

NB:

* bij het aanmaken van een nieuwe todo maakt  de server een nieuwe id aan.
* in plaats van put wordt soms ook patch gebruikt.
* voor een collection gebruiken we een meervoud; zie: https://restapitutorial.com/lessons/restfulresourcenaming.html (pluralization).
* we kunnen dit interface later uitbreiden met users.

### Idempotente opdrachten

We noemen een opdracht *idempotent* als deze bij herhaling hetzelfde effect blijft geven, met andere woorden: er is geen verschil tussen 1 keer of meer dan 1 keer uitvoeren.

Voorbeelden:

* `i = 100` is idempotent
* `i = i + 1` is *niet* idempotent
* `i = 100 * j + k` is idempotent
* `print(i)` is niet idempotent

Sommige HTTP-opdrachten zijn idempotent, andere niet:

* idempotent: `GET`, `PUT`, `DELETE`
* niet idempotent: `POST`

REST-API's zijn gebaseerd op deze eigenschappen. Voor veranderingen die niet idempotent zijn, zoals het aanmaken van een nieuwe todo, gebruiken we een niet-idempotente opdracht: `POST`.

> De browser maakt ook een onderscheid tussen idempotente opdrachten (`GET`) en niet-idempotente opdrachten (`POST`): het herladen van een normale webpagina (`GET`) gaat zonder vragen; bij het herladen van een pagina met een formulier (`POST`) krijg je de vraag of je het formulier nogmaals wilt opsturen.

## De serverzijde

Aan de kant van de server moeten we deze verschillende methods implementeren.
We slaan de todo's op een een eenvoudige SQL-database. 
We gebruiken SQLite als relational database management system.

Voor de todo's gebruiken we een enkele tabel:

```
CREATE TABLE Todos (id INTEGER PRIMARY KEY AUTOINCREMENT, text TEXT, done INTEGER);
```

* `id` is de key, een volgnummer in de database;
* `text` is de tekst van de todo
* `done` is een boolean: 0=not done, 1=done

(SQLite heeft geen boolean datatype.)


## Requests vanuit het browser URL-venster

Een URL die je invult in het browser URL-venster heeft een HTTP GET-opdracht als resultaat. 
Sommige van de bovengenoemde requests kun je dus ook in het URL-venster van de browser invullen, zoals (GET) `https://boom-wishbone.glitch.me/todos`.

Voor andere requests heb je HTTP-POST, PUT of DELETE nodig: die kun je niet vanuit het browser URL-venster invoeren. We kunnen daarvoor wel gebruik maken van Python.

## Requests vanuit Python

We testen het serverinterface uit met behulp van Python. We gebruiken hiervoor de `requests`-library.

In [ ]:
import requests

### Alle todo's

Een eenvoud voorbeeld: opvragen van alle todo's

In [ ]:
r = requests.get("https://boom-wishbone.glitch.me/todos")
print(r)
print("=======")
r.text

Het veld `r.text` bevat het resultaat-document. In dit geval is het een JSON-document met daarin alle todo's. 

In [ ]:
def all_todos():
    r = requests.get("https://boom-wishbone.glitch.me/todos")
    print(r)
    print("=======")
    print(r.text)    


In [ ]:
all_todos()

### Reset database

Voor de reset van de database gebruiken we de URL: ".../todos/reset".
Deze opdracht is handig bij het testen van de server: we kunnen bij de volgende opdrachten dan uitgaan van een bekende toestand.

In [ ]:
r1 = requests.post("https://boom-wishbone.glitch.me/todos/reset")
print(r1)
r1.text

In [ ]:
all_todos()

### Opvragen van een enkele todo

In [ ]:
r2 = requests.get("https://boom-wishbone.glitch.me/todos/1")
print(r2)
r2.text

### Aanmaken van een nieuwe todo

Bij het aanmaken van een nieuwe todo, via `POST /todos`, geven we als data de tekst van de todo mee. 
Deze data wordt als onderdeel van de POST-opdracht verstuurd naar de server.

Opmerking: `POST` is niet idempotent, dat wil zeggen: het maakt verschil of je deze eenmalig uitvoert, of vaker dan 1 keer. Als je de opdracht hieronder herhaalt, wordt steeds een nieuwe todo aangemaakt, met dezelfde tekst, maar met een andere key (volgnummer).

**Opdracht** Ga dit na.

In [ ]:
r3 = requests.post("https://boom-wishbone.glitch.me/todos", {"text": "afwas doen"})
print(r3)
r3.text

In [ ]:
all_todos()

### Aanpassen van een bestaande todo

Voor het aanpassen van een bestaande todo met als key 123 gebruiken we `PUT /todos/123`.

**N.B.** `PUT` is wel een *idempotente* opdracht: het maakt niet uit of je deze 1 maal of vaker uitvoert, dat geeft hetzelfde resultaat.

**Opdracht** Ga dit na.

In [ ]:
r5 = requests.put("https://boom-wishbone.glitch.me/todos/1", {"id":1,"text":"Make a todo app","done":1})
print(r5)
r5.text

In [ ]:
all_todos()

### Verwijderen van een todo

Voor het verwijderen van een todo met als key 123 gebruiken we `DELETE /todos/123`.

In [ ]:
r6 = requests.delete("https://boom-wishbone.glitch.me/todos/4")
print(r6)
r6.text

In [ ]:
all_todos()

## Server-code - node.js

### Anonieme functies

In de JavaScript-code voor de server kom je vaak de constructie `(req, resp) => {...}` tegen.
Dit is de notatie voor een anonieme functie met als parameters `(req, resp)` en als body (statements) `{...}`.
We noemen functies in deze notatie ook wel "arrow functions". Overal waar je zo'n `=>` tegenkomt is er dus sprake van een functie-definitie.

* https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Functions/Arrow_functions)

> Er is een enkel subtiel verschil met de anonieme functie `function (req, resp) {...}`, met betrekking tot `this`. Zie bovengenoemde verwijzing.

### Express

* zie: https://expressjs.com/en/starter/hello-world.html
* https://expressjs.com/en/guide/routing.html

`app.post("/todos/reset", (request, response)=> {...}`
          
Met deze expressie geef je aan:

* voor de request `POST /todos/reset`
* wordt de (anonieme) functie `(request, response) => {...}` uitgevoerd

Zoals je ziet kun je in deze anonieme functie de data uit het `request` gebruiken.
Deze functie moet eindigen met het versturen van een response naar de afzender van het request, bijvoorbeeld:
`response.send({ message: "success" });`

> Als je dit laatste vergeet, blijft de browser (client) wachten totdat er een time-out optreedt. Dit is niet waar de browser op zit te wachten.

`app.put("/todos/:id", (request, response) => {...}`

* de URL heeft een variabel deel `:id`
* deze `:id` is in de functie beschikbaar als `request.params.id`
* de data die meegestuurd is, is in de functie beschikbaar als `request.body.text`.

**Opdracht** Ga na wat de andere *routes* zijn in de todo-applicatie.

### Asynchrone verwerking, callbacks

De bovengenoemde vorm met een callback (hier als anonieme functie) komen we veel tegen bij opdrachten waarvan de verwerking *asynchroon* op een nader te bepalen tijdstip gebeurt. De normale verwerking gaat verder met de volgende opdracht.

* in het geval van de bovengenoemde routerings-opdrachten zet je de handlers voor de verschillende routes klaar. Deze worden pas uitgevoerd als er een request met een passende route (URL) verwerkt wordt.
* bij de database-opdrachten weet je niet hoe lang de verwerking van de opdracht duurt; je wilt daar niet op wachten. Als de database-opdracht klaar is, wordt de callback uitgevoerd.

### Database

De server slaat de todo's op in een (SQLite) database. Voor het opbergen en opzoeken van todo's kom je in de meeste functies één of meer database-opdrachten tegen.

* zie: sqlite.org
* https://www.sqlitetutorial.net/sqlite-nodejs/

De algemene vorm voor een database-opdracht is:

`db.run( SQL_opdracht, [...parameters], err => {...});`

De SQL-opdracht is een string met daarin 0 of meer `?`-s. 
De waarden die hiervoor ingevuld moeten worden staan in het array `[...parameters]`. 
Als de opdracht afgerond is wordt de anonieme functie `err => {...}` aangeroepen. De parameter `err` geeft aan of de opdracht succesvol is uitgevoerd, of niet.

> Sommige andere interfaces gebruiken twee functies, 1 voor success en 1 voor error. In dit interface zijn die gecombineerd.
